In [31]:
import requests
import pandas as pd

def load_data(symbol, start_date, end_date, interval):
    api_key = 'de6ee984b4c24a0c9c7a43d7dca8b75e' 
    order = 'asc'
    api_url = f'https://api.twelvedata.com/time_series?symbol={symbol}&start_date={start_date}&end_date={end_date}&interval={interval}&order={order}&apikey={api_key}'
        
    response = requests.get(api_url)
    if response.status_code != 200:
        raise Exception("Error in API request")
        
    data = response.json()
    
    # extract the data from the 'values' key
    data = pd.DataFrame(data['values'])
    data.dropna(inplace=True) # Drop NaN values
        
    # cleaning to numeric
    for col in data.columns[1:]: 
        data[col] = pd.to_numeric(data[col], errors='coerce')
        
    # save to CSV
    # data.to_csv(output_csv_path, index=False)
    # print(f"Data has been successfully saved to {output_csv_path}")
    
    return data

In [32]:
# first try : daily data

symbol = "BTC/USD"
start_date = "2023-01-01"
end_date = "2023-12-31"
interval = "1day"

df = load_data(symbol, start_date, end_date, interval)

In [33]:
df

,datetime,open,high,low,close
0,2023-01-01,16547.91406,16630.43945,16521.23438,16625.08008
1,2023-01-02,16625.50977,16759.34375,16572.22852,16688.47070
2,2023-01-03,16688.84766,16760.44727,16622.37109,16679.85742
3,2023-01-04,16680.20508,16964.58594,16667.76367,16863.23828
4,2023-01-05,16863.47266,16884.02148,16790.28320,16836.73633
...,...,...,...,...,...
360,2023-12-27,42518.46875,43683.16016,42167.58203,43442.85547
361,2023-12-28,43468.19922,43804.78125,42318.55078,42627.85547
362,2023-12-29,42614.64453,43124.32422,41424.06250,42099.40234
363,2023-12-30,42091.75391,42584.12500,41556.22656,42156.90234


In [34]:
# save the requested dataset to a new CSV
output_csv = "bitcoin_daily_2023_stock_data.csv"
df.to_csv(output_csv, index=False)

In [35]:
# new request : hourly data 

symbol = "BTC/USD"
start_date = "2023-01-01"
end_date = "2023-12-31"
interval = "1h"

df2 = load_data(symbol, start_date, end_date, interval)

In [47]:
df2 # starts at 2023-06-05 midday, because of request limit 

,datetime,open,high,low,close
0,2023-06-05 14:00:00,26728.42,26824.17,26706.32,26806.36
1,2023-06-05 15:00:00,26806.38,26813.96,25993.21,26011.01
2,2023-06-05 16:00:00,25998.49,26083.78,25461.05,25823.06
3,2023-06-05 17:00:00,25821.96,25823.71,25584.09,25763.99
4,2023-06-05 18:00:00,25763.98,25765.94,25526.57,25562.06
...,...,...,...,...,...
4995,2023-12-30 20:00:00,42345.91,42392.47,42194.28,42276.48
4996,2023-12-30 21:00:00,42276.49,42323.52,42213.61,42291.50
4997,2023-12-30 22:00:00,42291.24,42299.98,42059.83,42132.36
4998,2023-12-30 23:00:00,42132.40,42291.45,42093.90,42141.03


In [40]:
# second request for hourly data, ending on 2023-06-06 to cover the whole day of 06-05

symbol = 'BTC/USD'  # bitcoin
start_date = '2023-01-01'  
end_date = '2023-06-06'  
interval = "1h"

df3 = load_data(symbol, start_date, end_date, interval)

In [48]:
df3

,datetime,open,high,low,close
0,2023-01-01 00:00:00,16528.02930,16533.83984,16499.61914,16522.02930
1,2023-01-01 01:00:00,16520.26953,16546.01953,16519.13086,16544.56055
2,2023-01-01 02:00:00,16544.63086,16549.64062,16529.55078,16537.23047
3,2023-01-01 03:00:00,16537.97070,16538.59961,16511.92969,16524.18945
4,2023-01-01 04:00:00,16524.30078,16524.75977,16505.65039,16508.80078
...,...,...,...,...,...
3724,2023-06-05 20:00:00,25634.90000,25685.15000,25594.15000,25636.65000
3725,2023-06-05 21:00:00,25636.67000,25689.35000,25517.39000,25667.17000
3726,2023-06-05 22:00:00,25666.34000,25784.16000,25654.48000,25720.55000
3727,2023-06-05 23:00:00,25720.22000,25874.57000,25708.71000,25741.50000


In [49]:
combined_df = pd.concat([df3, df2])
combined_df.drop_duplicates(inplace=True)

In [50]:
combined_df

,datetime,open,high,low,close
0,2023-01-01 00:00:00,16528.02930,16533.83984,16499.61914,16522.02930
1,2023-01-01 01:00:00,16520.26953,16546.01953,16519.13086,16544.56055
2,2023-01-01 02:00:00,16544.63086,16549.64062,16529.55078,16537.23047
3,2023-01-01 03:00:00,16537.97070,16538.59961,16511.92969,16524.18945
4,2023-01-01 04:00:00,16524.30078,16524.75977,16505.65039,16508.80078
...,...,...,...,...,...
4995,2023-12-30 20:00:00,42345.91000,42392.47000,42194.28000,42276.48000
4996,2023-12-30 21:00:00,42276.49000,42323.52000,42213.61000,42291.50000
4997,2023-12-30 22:00:00,42291.24000,42299.98000,42059.83000,42132.36000
4998,2023-12-30 23:00:00,42132.40000,42291.45000,42093.90000,42141.03000


In [51]:
# save the combined dataset to a new CSV
combined_output_csv = "bitcoin_hourly_2023_stock_data.csv"
combined_df.to_csv(combined_output_csv, index=False)